# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846862674927                   -0.70    4.5         
  2   -7.852328695673       -2.26       -1.53    1.0   23.0ms
  3   -7.852616145052       -3.54       -2.55    1.5   24.9ms
  4   -7.852645961751       -4.53       -2.88    2.5   31.2ms
  5   -7.852646500904       -6.27       -3.18    1.0   23.0ms
  6   -7.852646678787       -6.75       -4.07    1.0   23.7ms
  7   -7.852646686684       -8.10       -5.05    2.2   31.2ms
  8   -7.852646686725      -10.39       -5.43    1.5   26.2ms
  9   -7.852646686728      -11.52       -5.57    1.5   26.1ms
 10   -7.852646686730      -11.69       -6.70    1.0   23.8ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846650334104                   -0.70           4.8         
  2   -7.852524320213       -2.23       -1.63   0.80    2.0    256ms
  3   -7.852635067408       -3.96       -2.74   0.80    1.0   21.6ms
  4   -7.852646582222       -4.94       -3.22   0.80    2.2   29.4ms
  5   -7.852646681389       -7.00       -4.06   0.80    1.0   21.9ms
  6   -7.852646686380       -8.30       -4.84   0.80    1.2   22.9ms
  7   -7.852646686719       -9.47       -6.03   0.80    2.0   26.4ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.365158e+01     3.203774e+00
 * time: 0.0571138858795166
     1     1.168081e+00     1.788360e+00
 * time: 0.27680301666259766
     2    -1.403232e+00     2.070497e+00
 * time: 0.2996499538421631
     3    -3.476088e+00     1.637649e+00
 * time: 0.33226895332336426
     4    -4.759202e+00     1.533888e+00
 * time: 0.3651309013366699
     5    -6.500367e+00     1.194463e+00
 * time: 0.3979198932647705
     6    -7.329846e+00     5.628575e-01
 * time: 0.43091893196105957
     7    -7.614910e+00     1.475561e-01
 * time: 0.4538588523864746
     8    -7.719996e+00     1.947586e-01
 * time: 0.4768340587615967
     9    -7.771750e+00     1.456097e-01
 * time: 0.49973583221435547
    10    -7.811256e+00     1.042340e-01
 * time: 0.5229129791259766
    11    -7.837440e+00     6.310637e-02
 * time: 0.5461008548736572
    12    -7.845147e+00     6.162088e-02
 * time: 0.5694029331207275
    13    -7.846466e+00     5.594612e-02
 * time: 0.592334

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846825624597                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645846837                   -1.64         
  2   -7.852646686730       -6.08       -3.71    1.94s
  3   -7.852646686730      -13.11       -7.19    165ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.6981494345650427e-7
|ρ_newton - ρ_scfv| = 9.8822215887817e-7
|ρ_newton - ρ_dm|   = 8.79071541876669e-10
